# Tensorflow Estimators

In [ ]:
import tensorflow as tf
import pandas as pd
assert "1.7" <= tf.__version__, "TensorFlow 1.7 or later is needed"

In [ ]:
df = pd.read_csv('../data/iris.csv')

In [ ]:
df.head()

In [ ]:
X = df.drop('target', axis='columns')
y = df['target']

column_names = df.columns[:-1]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
y_train.head()

In [ ]:
labels = ['setosa', 'versicolor', 'virginica']

In [ ]:
feature_columns = [
    tf.feature_column.numeric_column(key=key)
    for key in X_train.columns
]

In [ ]:
feature_columns

## Canned Estimators

In [ ]:
rm -rf ./models/iris/

In [ ]:
model = tf.estimator.DNNClassifier(
    hidden_units=[10, 10], 
    feature_columns=feature_columns,
    model_dir='./models/iris/DNN/',
    n_classes=3,
    label_vocabulary=labels,
    optimizer='Adam')

In [ ]:
batch_size=32
train_steps=3000

### Input function

In [ ]:
def input_fn(features, labels, batch_size, shuffle=True, repeat=None):
    feature_dict = dict(features)

    if labels is None:
        inputs = feature_dict
    else:
        inputs = (feature_dict, labels)
        
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    
    if shuffle:
        dataset = dataset.shuffle(1000)
    
    dataset = dataset.repeat(repeat)

    return dataset.batch(batch_size)

In [ ]:
def train_input_fn():
    return input_fn(X_train, y_train, batch_size)

### Model Training

In [ ]:
model.train(
    input_fn=train_input_fn,
    steps=3000)

In [ ]:
!ls {model_path}

### Evaluate model on train and test data

In [ ]:
train_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_train, y_train,
                             batch_size,
                             shuffle=False,
                             repeat=1))

test_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_test, y_test,
                             batch_size,
                             shuffle=False,
                             repeat=1))

In [ ]:
train_eval_result

In [ ]:
test_eval_result

### Predict on new data

In [ ]:
predict_x = {
    'sepal_length_cm': [5.1, 5.9, 6.9],
    'sepal_width_cm': [3.3, 3.0, 3.1],
    'petal_length_cm': [1.7, 4.2, 5.4],
    'petal_width_cm': [0.5, 1.5, 2.1],
}

In [ ]:
expected = ['setosa', 'versicolor', 'virginica']

In [ ]:
predictions = model.predict(
    input_fn=lambda:input_fn(predict_x,
                             labels=None,
                             batch_size=batch_size,
                             shuffle=False,
                             repeat=1))

In [ ]:
predictions

In [ ]:
template = ('\nPrediction is "{}", prob: {:.1f}%, expected "{}"')

for pred_dict, expec in zip(predictions, expected):
    
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    pred_label = labels[class_id]

    print(template.format(pred_label, 100 * probability, expec))

In [ ]:
pred_dict

## Exercise 1

The `tf.estimator` module offers 2 additional canned classifiers:

- `tf.estimator.BaselineClassifier`
- `tf.estimator.LinearClassifier`

What do they do?

Try substituting them in the model definition and re-run the trainin, evaluation and prediction steps.
Is the performance better or worse?

Note: you can set the logging level of tensorflow with `tf.logging.set_verbosity`

In [ ]:
tf.logging.set_verbosity(tf.logging.WARN)

#### Baseline Classifier

In [ ]:
model = tf.estimator.BaselineClassifier(
    model_dir='./models/iris/Baseline/',
    n_classes=3,
    label_vocabulary=labels)

In [ ]:
model.train(
    input_fn=train_input_fn,
    steps=3000)

In [ ]:
train_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_train, y_train,
                             batch_size,
                             shuffle=False,
                             repeat=1))

test_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_test, y_test,
                             batch_size,
                             shuffle=False,
                             repeat=1))

In [ ]:
train_eval_result

In [ ]:
test_eval_result

#### Linear Classifier

In [ ]:
model = tf.estimator.LinearClassifier(
    feature_columns=feature_columns,
    model_dir='./models/iris/Linear/',
    n_classes=3,
    label_vocabulary=labels)

In [ ]:
model.train(
    input_fn=train_input_fn,
    steps=3000)

In [ ]:
train_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_train, y_train,
                             batch_size,
                             shuffle=False,
                             repeat=1))

test_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_test, y_test,
                             batch_size,
                             shuffle=False,
                             repeat=1))

In [ ]:
train_eval_result

In [ ]:
test_eval_result

## Exercise 2

Visualize the training results with tensorboard:

- in a terminal, from the notebook folder, run tensorboard with `tensorboard --logdir=models`
- open the browser at [http://localhost:6006](http://localhost:6006)

